In [ ]:
from itertools import combinations
from z3 import *
import numpy as np


In [ ]:
width = 8
dims = ((3, 3),
            (3, 5),
            (5, 3),
            (5, 5))
print(dims)
max_height = int(np.sum([dims[i][1] for i in range(len(dims))]))
print(f"Maximum height reachable: {max_height}")

In [ ]:
def min_height(dims):
  return int(np.min([dims[i][1] for i in range(len(dims))]))

def compute_area_circuits(dims):
  return sum([dim[0]* dim[1] for dim in dims])


def vlsi_instance(dims, width, max_height):
  """
        Variable declaration
  """
  height = Int('height')
  corner_coords = [[Int("corner_coords_%s_%s" % (i + 1, j + 1)) for j in range(2)] for i in range(len(dims))]

  x = [[Bool(f"x_{i + 1}_{j+1}") for j in range(len(dims))] for i in range(len(dims))]
  y = [[Bool(f"y_{i + 1}_{j+1}") for j in range(len(dims))] for i in range(len(dims))]

  """
        Constraint definition
  """
    # board boundaries constraints
  boundaries_c_x = [And(0 <= corner_coords[i][0], corner_coords[i][0] + dims[i][0] <= width) for i in range(len(dims))]
  boundaries_c_y = [And(0 <= corner_coords[i][1], corner_coords[i][1] + dims[i][1] <= height) for i in range(len(dims))]
  height_constraint = [And(height < max_height, height > min_height(dims))]
 
  # no overlapping constraints

  no_overlapping = []
  for i in range(len(dims)):
    for j in range(len(dims)): # to fix
      if i != j:
        no_overlapping.append(
          If(corner_coords[i][0] < corner_coords[j][0],
                corner_coords[i][0] + dims[i][0] <= corner_coords[j][0],
          If(corner_coords[i][1] < corner_coords[j][1],
                corner_coords[i][1] + dims[i][1] <= corner_coords[j][1],
          If(corner_coords[i][0] > corner_coords[j][0],
                corner_coords[i][0] - dims[j][0] >=corner_coords[j][0],
          If(corner_coords[i][1] > corner_coords[j][1],
                corner_coords[i][1] - dims[j][1] >= corner_coords[j][1],
                False)))))

  # print(no_overlapping)
  all_constraints = height_constraint + boundaries_c_x  + boundaries_c_y + no_overlapping
  
  """
    Solving phase
  """
  solver = Optimize()
  solver.add(all_constraints)
  # minimizing function
  circuit_area = compute_area_circuits(dims)
  solver.minimize(width* height - circuit_area)
  if solver.check() == sat:
    m = solver.model()
    return m.evaluate(height), [[int(m.evaluate(corner_coords[i][j]).as_string()) for j in range(2)] for i in range(len(dims))]
  else:
    print("Failed to find a solution")
    return None


In [8]:
 vlsi_instance(dims, width, max_height)

NameError: name 'Int' is not defined